In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from google.colab import drive
from sklearn.model_selection import train_test_split
from sklearn import ensemble
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.preprocessing import MinMaxScaler
from google.colab import drive
import os
import warnings
warnings.filterwarnings("ignore")
pd.set_option("display.max_columns", None)
pd.set_option("display.width", None)

In [7]:
# Test the prediction Algorithym
output_dfs, signal_results, results_avg = test_algorythim()

# Print results for verification (positive ROI indicates profit)
print("Test Results: ")
print(f"Number of Stocks Analyzed: {len(output_dfs)}")
print(f"Number of 'BUY' Signals Over Analyzed Period: {len(signal_results)}")
print(f"ROI: {np.mean(signal_results):.2f}%")  # Changed line: Directly use signal_results
print(f'Number of Greens: {len([x for x in signal_results if x > 0])}')
print(f'Number of Reds: {len([x for x in signal_results if x < 0])}')
print(f'Green/Red Ratio: {(len([x for x in signal_results if x > 0])/len([x for x in signal_results if x < 0])):.2f}')
print('')
print('')

print(results_avg)



# Print names of dataframes for verification
for name, df in output_dfs.items():
    print(f"DataFrame name: {name}")
    print(f'ROI: {np.mean(results_avg[name]):.2f}%')
    print(df.head())
    print('')

Test Results: 
Number of Stocks Analyzed: 10
Number of 'BUY' Signals Over Analyzed Period: 434
ROI: -7.18%
Number of Greens: 120
Number of Reds: 314
Green/Red Ratio: 0.38


{'HistoricalData_Netflix': [-26.25025349023874, -24.86841743264921, -24.777281871341657, -22.98487140725879, -22.34927354430339, -21.52964317248813, -21.11847898642924, -24.019414274690284, -23.366240092686244, -22.3752033819314], 'HistoricalData_AMD': [-1.6146993318485476, 0.7204812534974825, -0.42081638378455793, 3.005474386397426, 0.6862889019377585, -0.6997084548104808, -3.3882848123465537, 0.4555484376112081, -0.993735147980122], 'HistoricalData_Tesla': [2.673187539944981, 1.3966005665722399, 1.0991758484268372, 1.6742852010347506, 2.701737681504416, -2.6067875950848522, -0.8123167155425166, -2.8954797472423883, -1.2128792065384997], 'HistoricalData_Amazon': [-7.2116977254420265, -7.200956901017554, -4.762359846959549, -4.603360380120771, -2.634306759712208, -3.098757064956452, -0.6269359541388091, 0.6561012834

In [2]:
def test_algorythim():
    base_path = '/content/drive/MyDrive/Análise de Dados/Estudo/Stock_Prediction/DataBases/'
    file_names = [
        'HistoricalData_Netflix',
        'HistoricalData_AMD',
        'HistoricalData_Tesla',
        'HistoricalData_Amazon',
        'HistoricalData_Meta',
        'HistoricalData_Qualcomm',
        'HistoricalData_Cisco',
        'HistoricalData_Microsoft',
        'HistoricalData_Starbucks',
        'HistoricalData_Apple'

    ]

    output_dataframes = {}
    number_of_signals = {}
    results_avg = {}
    signal_results_per_stock = []
    signal_results = []


    for file_name in file_names:
        file_path = os.path.join(base_path, file_name + '.csv')
        if os.path.exists(file_path):
            df = test_prediction_open_and_close_prices(file_path)
            output_dataframes[file_name] = df.copy()

            for df_name, df in output_dataframes.items(): # Iterar sobre cada DataFrame no dicionário
              signal_results_per_stock = []

              for index, row in df.iterrows():# Iterar sobre cada linha no DataFrame

                if row['Buy_Signal'] == 'BUY': # Verificar o valor da coluna 'Buy_Signal'
                    # Adicionar o valor da coluna 'ROI' à lista sample_results
                    signal_results.append(row['ROI'])
                    signal_results_per_stock.append(row['ROI'])

              results_avg[df_name] = signal_results_per_stock


        else:
            print(f"File {file_path} does not exist.")



    return output_dataframes, signal_results, results_avg



In [6]:
def test_prediction_open_and_close_prices(file_path):

    path = file_path
    df = pd.read_csv(path, encoding='unicode_escape') #Abrindo o arquivo .csv como um dataframe dentro do Python
    df['Date'] = pd.to_datetime(df['Date'])
    my_range = df['Date'].iloc[:10].tolist() # Criar a lista my_range com os primeiros 10 valores datetime da coluna "date"
    df = df.set_index('Date')
    predicao_day = pd.DataFrame(columns=['Predicted_Close', 'Real_Close', 'Predicted_Open', 'Real_Open', 'ROI', 'Buy_Signal'], index=my_range)
    profundidade = 100 + 1
    aimed_profit_margin = 0.02




    for i in my_range:

      daily_df = df.copy()

      for col in ['Open', 'High', 'Low', 'Close/Last']:
        # Remove dollar signs and commas before converting to float
        daily_df[col] = daily_df[col].str.replace('$', '').str.replace(',', '').astype(float)

      for profundidade in range(1,101): # Adicionar colunas com os valores dos dias anteriores
        daily_df[f'Prev_Close_{profundidade}'] = daily_df['Close/Last'].shift(profundidade)
        daily_df[f'Prev_Volume_{profundidade}'] = daily_df['Volume'].shift(profundidade)
        daily_df[f'Prev_Open_{profundidade}'] = daily_df['Open'].shift(profundidade)
        daily_df[f'Prev_High_{profundidade}'] = daily_df['High'].shift(profundidade)
        daily_df[f'Prev_Low_{profundidade}'] = daily_df['Low'].shift(profundidade)



      target_day = daily_df.loc[[i]] # Select the currently selected date as the target

      treino_day = daily_df.iloc[102:]

      x_treino_day_close = treino_day.drop(columns=['Open', 'High', 'Low', 'Close/Last', 'Volume'])


      y_treino_day_close = treino_day['Close/Last']


      x_treino_day_high = treino_day.drop(columns=['Open', 'High', 'Low', 'Close/Last', 'Volume'])


      y_treino_day_high = treino_day['High']


      x_treino_day_open = treino_day.drop(columns=['Open', 'High', 'Low', 'Close/Last', 'Volume'])


      y_treino_day_open = treino_day['Open']


      modelo_day_close = XGBRegressor().fit(x_treino_day_close, y_treino_day_close)


      modelo_day_high = XGBRegressor().fit(x_treino_day_high, y_treino_day_high)


      modelo_day_open = XGBRegressor().fit(x_treino_day_open, y_treino_day_open)


      test_day_close = target_day.drop(columns=['Open', 'High', 'Low', 'Close/Last', 'Volume'])
      #test_day_close = test_day_close[['Prev_Close', 'Prev_Volume', 'Prev_Open', 'Prev_High', 'Prev_Low']]#Select only the necessary columns


      test_day_high = target_day.drop(columns=['Open', 'High', 'Low', 'Close/Last', 'Volume'])
      #test_day_high = test_day_high[['Prev_Close', 'Prev_Volume', 'Prev_Open', 'Prev_High', 'Prev_Low']]#Select only the necessary columns


      test_day_open = target_day.drop(columns=['Open', 'High', 'Low', 'Close/Last', 'Volume'])
      #test_day_open = test_day_open[['Prev_Close', 'Prev_Volume', 'Prev_Open', 'Prev_High', 'Prev_Low']]#Select only the necessary columns



      predicao_day_close = modelo_day_close.predict(test_day_close)


      predicao_day_high = modelo_day_high.predict(test_day_high)


      predicao_day_open = modelo_day_open.predict(test_day_open)


      #Assign values using .loc with the current date i
      predicao_day.loc[i, 'Predicted_Close'] = predicao_day_close[0]
      predicao_day.loc[i, 'Real_Close'] = target_day['Close/Last'].values[0]
      predicao_day.loc[i, 'Predicted_High'] = predicao_day_high[0]
      predicao_day.loc[i, 'Real_High'] = target_day['High'].values[0]
      predicao_day.loc[i, 'Predicted_Open'] = predicao_day_open[0]
      predicao_day.loc[i, 'Real_Open'] = target_day['Open'].values[0]

      if predicao_day.loc[i, 'Real_High'] >= predicao_day.loc[i, 'Predicted_High']:

        predicao_day.loc[i, 'Sell_Price'] = predicao_day.loc[i, 'Predicted_High']
        predicao_day.loc[i, 'ROI'] = (predicao_day.loc[i, 'Sell_Price'] - predicao_day.loc[i, 'Real_Open']) / predicao_day.loc[i, 'Real_Open'] * 100
      else:
        predicao_day.loc[i, 'Sell_Price'] = predicao_day.loc[i, 'Real_Close']
        predicao_day.loc[i, 'ROI'] = (predicao_day.loc[i, 'Sell_Price'] - predicao_day.loc[i, 'Real_Open']) / predicao_day.loc[i, 'Real_Open'] * 100
      if predicao_day.loc[i, 'Predicted_High'] >= predicao_day.loc[i, 'Predicted_Open']*(1 + aimed_profit_margin):
          predicao_day.loc[i, 'Buy_Signal']= 'BUY'
      else:
          predicao_day.loc[i, 'Buy_Signal'] = 'DONT'



    return predicao_day

In [ ]:
def make_prediction_open_and_close_prices(file_path):

    path = file_path
    df = pd.read_csv(path, encoding='unicode_escape') #Abrindo o arquivo .csv como um dataframe dentro do Python
    df['Date'] = pd.to_datetime(df['Date'])
    df = df.set_index('Date')
    predicao_day = pd.DataFrame(columns=['Predicted_Close', 'Predicted_Open', 'Buy_Signal'])


    daily_df = df.copy()

    for col in ['Open', 'High', 'Low', 'Close/Last']:
      # Remove dollar signs and commas before converting to float
      daily_df[col] = daily_df[col].str.replace('$', '').str.replace(',', '').astype(float)


    treino_day = daily_df.copy() # training data

    # Extrair a última data do índice de daily_df
    last_date = treino_day.index[-1]

    # Calcular o próximo dia
    next_day = last_date + pd.Timedelta(days=1)

    # Criar o DataFrame target_day com o próximo dia como índice
    target_day = pd.DataFrame(index=[next_day])
    predicao_day.index = target_day.index


    x_treino_day_close = treino_day.drop(columns=['Close/Last'])


    y_treino_day_close = treino_day['Close/Last']


    x_treino_day_open = treino_day.drop(columns=['Open'])


    y_treino_day_open = treino_day['Open']


    modelo_day_close = XGBRegressor().fit(x_treino_day_close, y_treino_day_close)


    modelo_day_open = XGBRegressor().fit(x_treino_day_open, y_treino_day_open)


    target_day_close = target_day.drop(columns=['Close/Last'])


    test_day_open = target_day.drop(columns=['Open'])


    predicao_day_close = modelo_day_close.predict(test_day_close)


    predicao_day_open = modelo_day_open.predict(test_day_open)


    #Assign values using .loc with the current date i
    predicao_day['Predicted_Close'] = predicao_day_close[0]
    predicao_day['Real_Close'] = target_day['Close/Last'].values[0]
    predicao_day['Predicted_Open'] = predicao_day_open[0]
    predicao_day['Real_Open'] = target_day['Open'].values[0]

    predicao_day['ROI'] = (predicao_day['Real_Close'] - predicao_day['Real_Open']) / predicao_day['Real_Open'] * 100


    if predicao_day['Predicted_Close'] > predicao_day['Predicted_Open']:
      predicao_day['Buy_Signal']= 'BUY'
    else:
      predicao_day['Buy_Signal'] = 'DONT'



    return predicao_day

In [ ]:
path = '/content/drive/MyDrive/Análise de Dados/Estudo/Stock_Prediction/DataBases/HistoricalData_Netflix.csv'
df = pd.read_csv(path, encoding='unicode_escape') #Abrindo o arquivo .csv como um dataframe dentro do Python
df['Date'] = pd.to_datetime(df['Date'])
my_range = df['Date'].iloc[:10].tolist() # Criar a lista my_range com os primeiros 10 valores datetime da coluna "date"
df = df.set_index('Date')
predicao_day = pd.DataFrame(columns=['Predicted_Close', 'Real_Close', 'Predicted_Open', 'Real_Open','Predicted_High', 'Real_High', 'ROI', 'Buy_Signal'], index=my_range)
profundidade = 10 + 1
aimed_profit_margin = 0.02




for i in my_range:

  daily_df = df.copy()

  for col in ['Open', 'High', 'Low', 'Close/Last']:
    # Remove dollar signs and commas before converting to float
    daily_df[col] = daily_df[col].str.replace('$', '').str.replace(',', '').astype(float)

  for profundidade in range(1,11): # Adicionar colunas com os valores dos dias anteriores
    daily_df[f'Prev_Close_{profundidade}'] = daily_df['Close/Last'].shift(profundidade)
    daily_df[f'Prev_Volume_{profundidade}'] = daily_df['Volume'].shift(profundidade)
    daily_df[f'Prev_Open_{profundidade}'] = daily_df['Open'].shift(profundidade)
    daily_df[f'Prev_High_{profundidade}'] = daily_df['High'].shift(profundidade)
    daily_df[f'Prev_Low_{profundidade}'] = daily_df['Low'].shift(profundidade)



  #treino_day = daily_df.loc[i:] # Drop the currently selected date and all the previously selected dates from the training data

  target_day = daily_df.loc[[i]] # Select the currently selected date as the target

  treino_day = daily_df.iloc[11:]

  x_treino_day_close = treino_day.drop(columns=['Open', 'High', 'Low', 'Close/Last', 'Volume'])


  y_treino_day_close = treino_day['Close/Last']


  x_treino_day_high = treino_day.drop(columns=['Open', 'High', 'Low', 'Close/Last', 'Volume'])


  y_treino_day_high = treino_day['High']


  x_treino_day_open = treino_day.drop(columns=['Open', 'High', 'Low', 'Close/Last', 'Volume'])


  y_treino_day_open = treino_day['Open']


  modelo_day_close = XGBRegressor().fit(x_treino_day_close, y_treino_day_close)


  modelo_day_high = XGBRegressor().fit(x_treino_day_high, y_treino_day_high)


  modelo_day_open = XGBRegressor().fit(x_treino_day_open, y_treino_day_open)


  test_day_close = target_day.drop(columns=['Open', 'High', 'Low', 'Close/Last', 'Volume'])
  #test_day_close = test_day_close[['Prev_Close', 'Prev_Volume', 'Prev_Open', 'Prev_High', 'Prev_Low']]#Select only the necessary columns


  test_day_high = target_day.drop(columns=['Open', 'High', 'Low', 'Close/Last', 'Volume'])
  #test_day_high = test_day_high[['Prev_Close', 'Prev_Volume', 'Prev_Open', 'Prev_High', 'Prev_Low']]#Select only the necessary columns


  test_day_open = target_day.drop(columns=['Open', 'High', 'Low', 'Close/Last', 'Volume'])
  #test_day_open = test_day_open[['Prev_Close', 'Prev_Volume', 'Prev_Open', 'Prev_High', 'Prev_Low']]#Select only the necessary columns



  predicao_day_close = modelo_day_close.predict(test_day_close)


  predicao_day_high = modelo_day_high.predict(test_day_high)


  predicao_day_open = modelo_day_open.predict(test_day_open)


  #Assign values using .loc with the current date i
  predicao_day.loc[i, 'Predicted_Close'] = predicao_day_close[0]
  predicao_day.loc[i, 'Real_Close'] = target_day['Close/Last'].values[0]
  predicao_day.loc[i, 'Predicted_High'] = predicao_day_high[0]
  predicao_day.loc[i, 'Real_High'] = target_day['High'].values[0]
  predicao_day.loc[i, 'Predicted_Open'] = predicao_day_open[0]
  predicao_day.loc[i, 'Real_Open'] = target_day['Open'].values[0]
  #predicao_day.loc[i, 'Sell_Price'] = predicao_day.loc[i, 'Predicted_High']

  if predicao_day.loc[i, 'Real_High'] >= predicao_day.loc[i, 'Predicted_High']:

    predicao_day.loc[i, 'Sell_Price'] = predicao_day.loc[i, 'Predicted_High']
    predicao_day.loc[i, 'ROI'] = (predicao_day.loc[i, 'Sell_Price'] - predicao_day.loc[i, 'Real_Open']) / predicao_day.loc[i, 'Real_Open'] * 100
  else:
    predicao_day.loc[i, 'Sell_Price'] = predicao_day.loc[i, 'Real_Close']
    predicao_day.loc[i, 'ROI'] = (predicao_day.loc[i, 'Sell_Price'] - predicao_day.loc[i, 'Real_Open']) / predicao_day.loc[i, 'Real_Open'] * 100
  if predicao_day.loc[i, 'Predicted_High'] >= predicao_day.loc[i, 'Predicted_Open']*(1 + aimed_profit_margin):
      predicao_day.loc[i, 'Buy_Signal']= 'BUY'
  else:
      predicao_day.loc[i, 'Buy_Signal'] = 'DONT'

predicao_day.head(20)

,Predicted_Close,Real_Close,Predicted_Open,Real_Open,Predicted_High,Real_High,ROI,Buy_Signal,Sell_Price
2024-12-05,839.510986,917.87,812.193848,917.85,836.732666,927.0,-8.837755,BUY,836.732666
2024-12-04,838.926025,911.06,810.674072,898.05,832.536926,915.9699,-7.295036,BUY,832.536926
2024-12-03,827.212341,902.17,813.800354,894.36,825.438782,902.69,-7.706205,DONT,825.438782
2024-12-02,810.93158,897.74,808.987,887.51,815.913269,902.34,-8.067146,DONT,815.913269
2024-11-29,796.974731,886.81,800.486084,882.17,801.259338,892.41,-9.171777,DONT,801.259338
2024-11-27,794.328674,877.34,781.65033,872.06,782.139526,880.95,-10.311271,DONT,782.139526
2024-11-26,793.013855,872.6,785.166077,867.49,782.296631,886.36,-9.820674,DONT,782.296631
2024-11-25,793.996582,865.59,785.207825,902.05,780.704041,902.05,-13.452243,DONT,780.704041
2024-11-22,796.198425,897.79,779.210938,896.0,797.718201,906.0,-10.968951,BUY,797.718201
2024-11-21,828.290771,897.48,786.978088,883.93,823.992493,908.0,-6.780798,BUY,823.992493


In [ ]:
treino_day.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2505 entries, 2024-11-19 to 2014-12-08
Data columns (total 55 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Close/Last      2505 non-null   float64
 1   Volume          2505 non-null   int64  
 2   Open            2505 non-null   float64
 3   High            2505 non-null   float64
 4   Low             2505 non-null   float64
 5   Prev_Close_1    2505 non-null   float64
 6   Prev_Volume_1   2505 non-null   float64
 7   Prev_Open_1     2505 non-null   float64
 8   Prev_High_1     2505 non-null   float64
 9   Prev_Low_1      2505 non-null   float64
 10  Prev_Close_2    2505 non-null   float64
 11  Prev_Volume_2   2505 non-null   float64
 12  Prev_Open_2     2505 non-null   float64
 13  Prev_High_2     2505 non-null   float64
 14  Prev_Low_2      2505 non-null   float64
 15  Prev_Close_3    2505 non-null   float64
 16  Prev_Volume_3   2505 non-null   float64
 17  Prev_Open_3    

In [ ]:
treino_day.tail()

In [ ]:
treino_day.head()

,Close/Last,Volume,Open,High,Low,Prev_Close_1,Prev_Volume_1,Prev_Open_1,Prev_High_1,Prev_Low_1,Prev_Close_2,Prev_Volume_2,Prev_Open_2,Prev_High_2,Prev_Low_2,Prev_Close_3,Prev_Volume_3,Prev_Open_3,Prev_High_3,Prev_Low_3,Prev_Close_4,Prev_Volume_4,Prev_Open_4,Prev_High_4,Prev_Low_4,Prev_Close_5,Prev_Volume_5,Prev_Open_5,Prev_High_5,Prev_Low_5,Prev_Close_6,Prev_Volume_6,Prev_Open_6,Prev_High_6,Prev_Low_6,Prev_Close_7,Prev_Volume_7,Prev_Open_7,Prev_High_7,Prev_Low_7,Prev_Close_8,Prev_Volume_8,Prev_Open_8,Prev_High_8,Prev_Low_8,Prev_Close_9,Prev_Volume_9,Prev_Open_9,Prev_High_9,Prev_Low_9,Prev_Close_10,Prev_Volume_10,Prev_Open_10,Prev_High_10,Prev_Low_10
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2024-11-19,346.00,88852450,335.76,347.3799,332.75,342.03,66340650.0,345.00,346.5999,334.30,339.64,58011720.0,343.81,347.9899,335.28,352.56,89140720.0,341.085,361.5300,337.70,338.59,95890900.0,360.140,361.9300,338.20,338.23,62295860.0,341.000,346.9600,335.66,332.89,57896440.0,341.800,342.5500,326.59,345.16,37167620.0,336.080,345.45,334.6501,357.09,77986480.0,352.38,360.00,351.1501,351.42,58267200.0,351.80,355.69,348.2000,357.93,50810870.0,353.00,358.10,348.6000
2024-11-18,338.74,126547500,340.73,348.5499,330.01,346.00,88852450.0,335.76,347.3799,332.75,342.03,66340650.0,345.00,346.5999,334.30,339.64,58011720.0,343.810,347.9899,335.28,352.56,89140720.0,341.085,361.5300,337.70,338.59,95890900.0,360.140,361.9300,338.20,338.23,62295860.0,341.000,346.9600,335.66,332.89,57896440.0,341.800,342.55,326.5900,345.16,37167620.0,336.08,345.45,334.6501,357.09,77986480.0,352.38,360.00,351.1501,351.42,58267200.0,351.80,355.69,348.2000
2024-11-15,320.72,114440300,310.57,324.6799,309.22,338.74,126547500.0,340.73,348.5499,330.01,346.00,88852450.0,335.76,347.3799,332.75,342.03,66340650.0,345.000,346.5999,334.30,339.64,58011720.0,343.810,347.9899,335.28,352.56,89140720.0,341.085,361.5300,337.70,338.59,95890900.0,360.140,361.9300,338.20,338.23,62295860.0,341.000,346.96,335.6600,332.89,57896440.0,341.80,342.55,326.5900,345.16,37167620.0,336.08,345.45,334.6501,357.09,77986480.0,352.38,360.00,351.1501
2024-11-14,311.18,120726100,327.69,329.9800,310.37,320.72,114440300.0,310.57,324.6799,309.22,338.74,126547500.0,340.73,348.5499,330.01,346.00,88852450.0,335.760,347.3799,332.75,342.03,66340650.0,345.000,346.5999,334.30,339.64,58011720.0,343.810,347.9899,335.28,352.56,89140720.0,341.085,361.5300,337.70,338.59,95890900.0,360.140,361.93,338.2000,338.23,62295860.0,341.00,346.96,335.6600,332.89,57896440.0,341.80,342.55,326.5900,345.16,37167620.0,336.08,345.45,334.6501
2024-11-13,330.24,125405600,335.85,344.5999,322.50,311.18,120726100.0,327.69,329.9800,310.37,320.72,114440300.0,310.57,324.6799,309.22,338.74,126547500.0,340.730,348.5499,330.01,346.00,88852450.0,335.760,347.3799,332.75,342.03,66340650.0,345.000,346.5999,334.30,339.64,58011720.0,343.810,347.9899,335.28,352.56,89140720.0,341.085,361.53,337.7000,338.59,95890900.0,360.14,361.93,338.2000,338.23,62295860.0,341.00,346.96,335.6600,332.89,57896440.0,341.80,342.55,326.5900
